In [ ]:
import tensorflow as tf
import cv2
from models.base_learner import LearnerResnet, LearnerEigen
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
config = {
    'ds': 'uvd',    
    'use_eigen': False,
    'rgb_dir': './data/uvd/test/09/Images/',
    'depth_dir': './data/uvd/test/09/DepthSR/'
}
config_resnet = {
    'checkpoint_file': './experiments/resnet50_0306/model-40',
    'input_width': 304,
    'input_height': 228,
    'output_width': 304,
    'output_height': 228,
    'l2_reg_scale': 0.0
}
config_eigen = {
    'checkpoint_file': './tests_uvd/eigen_0106/model-130',
    'input_width': 304,
    'input_height': 228,
    'output_width': 74,
    'output_height': 55,
    'l2_reg_scale': 0.0
}

plt.rcParams['figure.figsize'] = (10.0, 7.0) # set default size of plots
font = {'family' : 'sans',
        'weight' : 'normal',
        'size'   : 10}
matplotlib.rc('font', **font)

In [ ]:
#---------------------#
# Define utilities & metrics
#---------------------#
def cvt2depth(img):
    scaling = 40.0/255 #factor to convert intensity to m
    return img.astype(float)*scaling
def diff (pred, target):
    return np.abs(pred-target)
def calc_rmse(pred, target):
    d = diff(cvt2depth(pred), cvt2depth(target))
    return np.sqrt(np.mean(d**2))
def calc_logrmse(pred, target):
    pred = cvt2depth(np.clip(pred,1e-4,float('inf')))
    target = cvt2depth(np.clip(target,1e-4,float('inf')))
    log_d = diff(np.log(pred), np.log(target))
    return np.sqrt(np.mean(log_d**2))
def calc_rel(pred, target):
    d = diff(cvt2depth(pred), cvt2depth(target))
    target = cvt2depth(target)
    return np.mean(d/target)
def calc_simse(pred, target):
    pred = cvt2depth(np.clip(pred,1e-4,float('inf')))
    target = cvt2depth(np.clip(target,1e-4,float('inf')))
    log_d = diff(np.log(pred), np.log(target))
    return np.mean(log_d**2)-np.mean(log_d)**2
def calc_d125(pred, target):
    d_lower = cvt2depth(pred)/cvt2depth(target)
    d_upper = cvt2depth(target)/cvt2depth(pred)
    inlier = pred[np.logical_and(d_lower<1.25, d_upper<1.25)]
    return float(inlier.size)/float(target.size)

In [ ]:
if config['use_eigen']:
    #---------------------#
    # Restore Eigen model
    #---------------------#
    tf.reset_default_graph()
    gEigen = tf.Graph()
    with gEigen.as_default():
        learnerEigen = LearnerEigen()
        learnerEigen.setup_inference(config_eigen)
        saverEigen = tf.train.Saver()
        sessEigen = tf.Session()
        saverEigen.restore(sessEigen, config_eigen['checkpoint_file'])

In [ ]:
#---------------------#
# Restore Resnet model
#---------------------#
tf.reset_default_graph()
gResnet = tf.Graph()
with gResnet.as_default():
    learnerResnet = LearnerResnet()
    learnerResnet.setup_inference(config_resnet)
    
    # Restore all found variables
    reader = tf.train.NewCheckpointReader(config_resnet['checkpoint_file'])
    checkpoint_shapes = reader.get_variable_to_shape_map()
    checkpoint_names = set(checkpoint_shapes.keys())
    model_names = set([v.name.split(':')[0] for v in tf.global_variables()])
    found_names = model_names & checkpoint_names
    found_vars = []
    lost_vars = model_names - checkpoint_names
    with tf.variable_scope('', reuse=True):
        for name in found_names:
            var = tf.get_variable(name)
            found_vars.append(var)
        print('Restored %d variables' % len(found_names))
    saverResnet = tf.train.Saver(var_list=found_vars)
    
    sessResnet = tf.Session()
    saverResnet.restore(sessResnet, config_resnet['checkpoint_file'])

In [ ]:
#---------------------#
# Plot UVD results 
# this function will plot 3 rows of images and depth predictions
# 1st image: first_frame
# 2nd image: first_frame+1*frame_spacing
# 2nd image: first_frame+2*frame_spacing
#---------------------#
first_frame=100
frame_spacing=500

n_rows=3
if config['use_eigen']:
    n_cols=4
else:
    n_cols=3

fig = plt.figure(figsize=(20,n_rows*4))
fig_row=1
# Only even img_numbers are allowed
for img_nr in range(2, 2+n_rows*frame_spacing, frame_spacing):
    filename_rgb = os.path.join(config['rgb_dir'], "%04d"%img_nr + ' 02.png')
    filename_depth = os.path.join(config['depth_dir'], "%04d"%img_nr + ' 02.png')
    image_rgb = cv2.cvtColor(cv2.imread(filename_rgb), cv2.COLOR_BGR2RGB)
    image_depth = cv2.imread(filename_depth, cv2.IMREAD_GRAYSCALE)
    
    inputs = {}
    inputs['rgb'] = np.expand_dims(image_rgb, 0)
    inputs['depth'] = np.expand_dims(np.expand_dims(image_depth, 0), -1)

    resultsResnet = learnerResnet.inference(inputs, sessResnet)
    pred_resnet = np.squeeze(resultsResnet['depth_prediction'])

    a1 = fig.add_subplot(3,n_cols,fig_row)
    a1.axis("off")
    a1.imshow(image_rgb, cmap='jet')
    a2 = fig.add_subplot(3,n_cols,fig_row+1)
    a2.axis("off")
    a2.imshow(image_depth, cmap='jet')
    a3 = fig.add_subplot(3,n_cols,fig_row+2)
    a3.axis("off")
    a3.imshow(pred_resnet, cmap='jet')
    a1.set_title('RGB')
    a2.set_title('Target')
    a3.set_title('Ours')

    if config['use_eigen']:
        resultsEigen = learnerEigen.inference(inputs, sessEigen)
        pred_eigen = np.squeeze(resultsEigen['depth_prediction'])
        a4 = fig.add_subplot(3,4,fig_row+3)
        a4.axis("off")
        a4.imshow(pred_eigen)
        a4.set_title('Eigen et al.')
    else:
        fig_row+=n_cols

fig.tight_layout()
#out_path = '/home/rpg_students/moritz/figures/depth_predictions'
#plt.savefig(os.path.join(out_path, config['ds']+'-DepthPredictions-'+ str(n)+'.png'), bbox_inches='tight', dpi=150)

In [ ]:
#---------------------#
# Calculate performance metrics
#---------------------#
rmse_resnet = []
rel_resnet = []
simse_resnet = []
logrmse_resnet = []
d125_resnet = []

rmse_eigen = []
rel_eigen = []
simse_eigen = []
logrmse_eigen = []
d125_eigen = []

n_images = 4500
sample_spacing = 100
# Only even img_numbers are allowed
for img_nr in range(2, 4500, sample_spacing):
    filename_rgb = os.path.join(config['rgb_dir'], "%04d"%img_nr + ' 02.png')
    filename_depth = os.path.join(config['depth_dir'], "%04d"%img_nr + ' 02.png')
    image_rgb = cv2.cvtColor(cv2.imread(filename_rgb), cv2.COLOR_BGR2RGB)
    image_depth = cv2.imread(filename_depth, cv2.IMREAD_GRAYSCALE)
    
    inputs = {}
    inputs['rgb'] = np.expand_dims(image_rgb, 0)
    inputs['depth'] = np.expand_dims(np.expand_dims(image_depth, 0), -1)

    resultsResnet = learnerResnet.inference(inputs, sessResnet)
    pred_resnet = np.squeeze(resultsResnet['depth_prediction'])
    
    rmse_resnet.append(calc_rmse(pred_resnet, image_depth))
    logrmse_resnet.append(calc_logrmse(pred_resnet, image_depth))
    rel_resnet.append(calc_rel(pred_resnet, image_depth))
    simse_resnet.append(calc_simse(pred_resnet, image_depth))
    d125_resnet.append(calc_d125(pred_resnet, image_depth))
    
    if config['use_eigen']:
        pred_eigen = np.squeeze(resultsEigen['depth_prediction'])
        resultsEigen = learnerEigen.inference(inputs, sessEigen)

        rmse_eigen.append(calc_rmse(pred_eigen, image_depth))
        logrmse_eigen.append(calc_logrmse(pred_eigen, image_depth))
        rel_eigen.append(calc_rel(pred_eigen, image_depth))
        simse_eigen.append(calc_simse(pred_eigen, image_depth))
        d125_eigen.append(calc_d125(pred_eigen, image_depth))

# Average performance over all samples
avg_rmse_resnet = np.mean(rmse_resnet)
avg_rel_resnet = np.mean(rel_resnet)
avg_simse_resnet = np.mean(simse_resnet)
avg_logrmse_resnet = np.mean(logrmse_resnet)
avg_d125_resnet = np.mean(d125_resnet)

print("N samples: %d" % len(rmse_resnet))
if config['use_eigen']:
    avg_rmse_eigen = np.mean(rmse_eigen)
    avg_rel_eigen = np.mean(rel_eigen)
    avg_simse_eigen = np.mean(simse_eigen)
    avg_logrmse_eigen = np.mean(logrmse_eigen)
    avg_d125_eigen = np.mean(d125_eigen)
    print("Performance(Resnet/Eigen) - Rel: %f/%f, RMSE: %f/%f, logRMSE: %f/%f, SIMSE: %f/%f, D1.25: %f/%f" 
      % (avg_rel_resnet, avg_rel_eigen, avg_rmse_resnet, avg_rmse_eigen, avg_logrmse_resnet, avg_logrmse_eigen, 
         avg_simse_resnet, avg_simse_eigen, avg_d125_resnet, avg_d125_eigen))
else:
    print("Performance(Resnet) - Rel: %f, RMSE: %f, logRMSE: %f, SIMSE: %f, D1.25: %f" 
      % (avg_rel_resnet, avg_rmse_resnet, avg_logrmse_resnet, avg_simse_resnet, avg_d125_resnet))

In [ ]:
#---------------------#
# Plot performance history over test split
#---------------------#
fig = plt.figure()
a1 = fig.add_subplot(2,3,1)
a1.set_title('Rel error')
a1.plot(rel_resnet)
a2 = fig.add_subplot(2,3,2)
a2.set_title('RMSE')
a2.plot(rmse_resnet)
a3 = fig.add_subplot(2,3,3)
a3.set_title('logRMSE')
a3.plot(logrmse_resnet)
a4 = fig.add_subplot(2,3,4)
a4.set_title('SI MSE')
a4.plot(simse_resnet)
a5 = fig.add_subplot(2,3,5)
a5.set_title('D1.25')
a5.plot(d125_resnet)